In [15]:
from python_speech_features import mfcc, delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import numpy as np

In [16]:
# Extract si mfcc dari file path
def extract_mfcc(file_path):
  rate,signal = wav.read(file_path)
  mfcc_features = mfcc(signal, rate, numcep=13)
  mfcc_delta = delta(mfcc_features, 2)
  mfcc_delta_delta = delta(mfcc_delta, 2)
  return np.hstack((mfcc_features, mfcc_delta, mfcc_delta_delta))


In [17]:
# Hitung jarak dari 1 test ke 1 template
def calculate_dtw(template, test):
  distance, path = fastdtw(template, test, dist=euclidean)
  return distance

In [18]:
# Cocokin 1 data test ke templates
def speech_recognition(test, templates):
  distances = []
  for vowel, template in templates.items():
    distance = calculate_dtw(template, test)
    distances.append((vowel, distance))
  recognized_vowel = min(distances, key=lambda x: x[1])[0]
  return recognized_vowel

In [19]:
def evaluate_accuracy(test_data, templates):
    correct = 0
    total = 0
    
    for test_vowel, test_mfcc in test_data.items():
       
        recognized_vowel = speech_recognition(test_mfcc, templates)
        print("Ini recognized_vowel dan test_vowel", recognized_vowel, test_vowel)
        if recognized_vowel == test_vowel:
            correct += 1
        total += 1
    
    accuracy = (correct / total) * 100
    return accuracy


In [27]:
template_folder = "template-wav/"
test_folder = "test-wav/"

templates = {
    'a': extract_mfcc(template_folder + "a.wav"),
    'b': extract_mfcc(template_folder + "b.wav"),
    'c': extract_mfcc(template_folder + "c.wav"),
    'd': extract_mfcc(template_folder + "d.wav"),
    'e': extract_mfcc(template_folder + 'e.wav')
}

test_data = {
    'a': extract_mfcc(test_folder + "a.wav"),
    'b': extract_mfcc(test_folder + "b.wav"),
    'c': extract_mfcc(test_folder + "c.wav"),
    'd': extract_mfcc(test_folder + "d.wav"),
    'e': extract_mfcc(test_folder + 'e.wav')
}
evaluate_accuracy(test_data, templates)


Ini recognized_vowel dan test_vowel a a
Ini recognized_vowel dan test_vowel d b
Ini recognized_vowel dan test_vowel e c
Ini recognized_vowel dan test_vowel d d
Ini recognized_vowel dan test_vowel e e


60.0